#### **Install các thư viện cần thiết**

In [ ]:
!pip install scrapy
!pip install spider3

#### **Import packages**

In [ ]:
import json
import re
import pandas as pd
import scrapy

#### **Tạo project với scrapy**

In [ ]:
!scrapy startproject laptopmedia

#### **Viết class để parse url của từng trang**

In [ ]:
import scrapy
import re 

class collect_laptop_url(scrapy.Spider):
  name='laptop_urls' 
  custom_settings = {
        'CONCURRENT_REQUESTS_PER_DOMAIN': 1,
        'DOWNLOAD_DELAY': 1
        }
  handle_httpstatus_list = [403]
  
  def start_requests(self):
    urls = ['https://laptopmedia.com/specs/?q=&hPP=1000&idx=laptops&p=0']
    yield scrapy.Request(url=urls[0], callback = self.parse)
  def parse(self, response):
    for href in response.xpath('//*[@id="results_container"]/div/div/article/a[1]/@href').extract():
        yield {
          'laptop_url': href
        }
    offset = re.findall(r'p=(\d+)', response.url)[0]
    if int(offset) < 20:
      next_url = 'https://laptopmedia.com/specs/?q=&hPP=20&idx=laptops&p=' + str(int(offset)+1) 
      yield response.follow(next_url, callback = self.parse, dont_filter=True)

In [ ]:
!pwd

In [ ]:
!cd laptopmedia/laptopmedia
!scrapy crawl laptop_urls -o dataset/laptop_urls.json

In [ ]:
import requests
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from lxml import html
import json
import httpx
import urllib.request
from time import sleep

browser = webdriver.Chrome(service=Service(ChromeDriverManager().install())) #webdriver.Chrome("chromedriver.exe")
browser.maximize_window()
browser.implicitly_wait(10)

url = 'https://laptopmedia.com/specs/'
browser.get(url)

wait = WebDriverWait(browser, 20)

def find_by_xpath(element_source,xpath_expression):
    root = html.fromstring(element_source)
    return root.xpath(xpath_expression)

# for href in find_by_xpath(browser.page_source, f'/html/body/header/div/div[4]/nav/div/ul/li[position()>1]/@href'):
#     print(href)

# HEADERS = {'User-Agent': 'Mozilla/5.0 (iPad; CPU OS 12_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148'}
# page = httpx.get('https://laptopmedia.com/specs/', headers=HEADERS)#.json()
wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="results_container"]'))) 
tree = html.fromstring(browser.page_source)

link = tree.xpath('//div[@id="results_container"]/div/div/article/a[1]/@href/text()')
print(link)

# fp = urllib.request.urlopen("https://laptopmedia.com/specs/")
# mybytes = fp.read()

# mystr = mybytes.decode("utf8")
# fp.close()

# print(mystr)


In [76]:
import pandas as pd
def url_generator(page_path, cat_id, page):
    return f"https://tiki.vn/api/personalish/v1/blocks/listings?limit=100&is_mweb=1&aggregations=2&sort=top_seller&urlKey={page_path}&category={cat_id}&page={page}"

page = httpx.get(url_generator('sach_truyen_tieng_viet', '316', '1')).json()
for i in page['filters']:
    if i['display_name'] == 'Danh Mục Sản Phẩm':
        categories = i['values']
       
# for index, i in enumerate(categories):
#     page = httpx.get(url_generator(i['url_key'], i['query_value'], '1')).json()

page = httpx.get(url_generator('sach_van_hoc', '839', '1')).json()
df = pd.DataFrame(page['data'])
display(df)


,id,sku,name,url_key,url_path,type,author_name,book_cover,brand_name,short_description,...,quantity_sold,video_url,tiki_live,original_price,shippable,impression_info,advertisement,availability,primary_category_path,product_reco_score
0,74021317,9653845109288,Cây Cam Ngọt Của Tôi,cay-cam-ngot-cua-toi-p74021317,cay-cam-ngot-cua-toi-p74021317.html?spid=74021318,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 51582}",None,None,108000,True,[{'impression_id': 'thanos-product-76j3tPRbuIl...,None,1,1/2/8322/316/839/844,0
1,109017985,7255202170163,"Thiên Tài Bên Trái, Kẻ Điên Bên Phải (Tái Bản)",thien-tai-ben-trai-ke-dien-ben-phai-tai-ban-p1...,thien-tai-ben-trai-ke-dien-ben-phai-tai-ban-p1...,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 20073}",None,None,179000,True,[{'impression_id': 'thanos-product-YSmfJ4FxPkj...,None,1,1/2/8322/316/839/845,0
2,52789367,6624824359331,Nhà Giả Kim (Tái Bản 2020),nha-gia-kim-tai-ban-2020-p52789367,nha-gia-kim-tai-ban-2020-p52789367.html?spid=5...,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 24046}",None,None,79000,True,[{'impression_id': 'thanos-product-oneM4EcUctx...,None,1,1/2/8322/316/839/842,0
3,200780130,3944554367561,Người bà tài giỏi vùng Saga – Tập 9,nguoi-ba-tai-gioi-vung-saga-tap-9-p200780130,nguoi-ba-tai-gioi-vung-saga-tap-9-p200780130.h...,,,None,,,...,"{'text': 'Đã bán 935', 'value': 935}",None,None,52000,True,[{'impression_id': 'thanos-product-0ijFhMGXMeu...,None,1,1/2/8322/316/839/8973,0
4,117254517,5222628547911,Đại Dương Đen - Những Câu Chuyện Từ Thế Giới C...,dai-duong-den-p117254517,dai-duong-den-p117254517.html?spid=117254521,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 9309}",None,None,240000,True,[{'impression_id': 'thanos-product-kA6C3nZS7Pc...,None,1,1/2/8322/316/839/845,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,70016692,6701145185686,Bắt Trẻ Đồng Xanh (Tái Bản 2020),bat-tre-dong-xanh-tai-ban-2020-p70016692,bat-tre-dong-xanh-tai-ban-2020-p70016692.html?...,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 6476}",None,None,88000,True,[{'impression_id': 'thanos-product-VZwkhwlYzgo...,None,1,1/2/8322/316/839/844,0
96,8886001,6743728276916,Giết Con Chim Nhại (Tái Bản),giet-con-chim-nhai-tai-ban-p8886001,giet-con-chim-nhai-tai-ban-p8886001.html?spid=...,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 12054}",None,None,120000,True,[{'impression_id': 'thanos-product-MIkI36SpGxG...,None,1,1/2/8322/316/839/6750,0
97,545278,7951162516621,Combo Sherlock Holmes Toàn Tập (Trọn Bộ 3 Tập),combo-sherlock-holmes-toan-tap-tron-bo-3-tap-p...,combo-sherlock-holmes-toan-tap-tron-bo-3-tap-p...,,,None,,,...,"{'text': 'Đã bán 5000+', 'value': 6971}",None,None,300000,True,[{'impression_id': 'thanos-product-IW1ODZbuVJS...,None,1,1/2/8322/316/839/664,0
98,193206168,2589535338336,Quán Cơm Tỳ Hưu Chỉ Có Vào Không Có Ra (Combo ...,quan-com-ty-huu-chi-co-vao-ko-co-ra-combo-2-cu...,quan-com-ty-huu-chi-co-vao-ko-co-ra-combo-2-cu...,,,None,,,...,"{'text': 'Đã bán 160', 'value': 160}",None,None,319000,True,[{'impression_id': 'thanos-product-oOWisagnniS...,None,1,1/2/8322/316/839/1367,0
